# CRUD operations with queue

## Setup

Import [azure data tables](https://pypi.org/project/azure-data-tables/) (version 12.2.0 Updated nov.2021)

> pip install azure-data-tables

INSERT ENVIRONMENT VARIABLES

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
conn_str = os.getenv("YOUR_CONNECTION_STRING") # AZURE_STORAGE_TABLE
q_name = "test"

from azure.storage.queue import (
        QueueClient,
        BinaryBase64EncodePolicy,
        BinaryBase64DecodePolicy
)

## Functions

#### Create a new queue

In [4]:
# Instantiate a QueueClient object which will
# be used to create and manipulate the queue
print("Creating queue: " + q_name)
queue_client = QueueClient.from_connection_string(conn_str, q_name)

# Create the queue
queue_client.create_queue()

Creating queue: test2


{'x-ms-request-id': 'e226ea0b-1003-003f-3c9d-2fb787000000',
 'x-ms-version': '2018-03-28',
 'Date': datetime.datetime(2022, 3, 4, 7, 54, 9, tzinfo=datetime.timezone.utc)}

Azure queue messages are stored as text. If you want to store binary data, setup Base64 encoding and decoding functions before putting a message in the queue.

Configure Base64 encoding and decoding functions when creating the client object.

In [ ]:
# Setup Base64 encoding and decoding functions
base64_queue_client = QueueClient.from_connection_string(
                            conn_str=conn_str, queue_name=q_name,
                            message_encode_policy = BinaryBase64EncodePolicy(),
                            message_decode_policy = BinaryBase64DecodePolicy()
                        )

### Insert a message into the queue

In [5]:
message = u"Hello World"
print("Adding message: " + message)
queue_client.send_message(message)


Adding message: Hello World


{'id': '1c1d92cd-7a93-4a51-bb82-b3440e7a7957', 'inserted_on': datetime.datetime(2022, 3, 4, 7, 57, 5, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2022, 3, 11, 7, 57, 5, tzinfo=datetime.timezone.utc), 'dequeue_count': None, 'content': 'Hello World', 'pop_receipt': 'AgAAAAMAAAAAAAAArLm0cJ0v2AE=', 'next_visible_on': datetime.datetime(2022, 3, 4, 7, 57, 5, tzinfo=datetime.timezone.utc)}

### Peek at message without removing it

In [6]:
# Peek at the first message
messages = queue_client.peek_messages()

for peeked_message in messages:
    print("Peeked message: " + peeked_message.content)

Peeked message: Hello World


### Change the contents of a queued message

You can change the contents of a message in-place in the queue. If the message represents a task, you can use this feature to update the status of the task.

In [7]:
messages = queue_client.receive_messages()
list_result = next(messages)

message = queue_client.update_message(
        list_result.id, list_result.pop_receipt,
        visibility_timeout=0, content=u'Hello World Again')

print("Updated message to: " + message.content)

Updated message to: Hello World Again


### Get the queue length
You can get an estimate of the number of messages in a queue.

PS! The result is only approximate because messages can be added or removed after the service responds to your request.

In [10]:
properties = queue_client.get_queue_properties()
count = properties.approximate_message_count
print("Message count: " + str(count))

Message count: 1


### Dequeue messages
Remove a message from a queue in two steps. If your code fails to process a message, this two-step process ensures that you can get the same message and try again. Call delete_message after the message has been successfully processed.

When you call receive_messages, you get the next message in the queue by default. A message returned from receive_messages becomes invisible to any other code reading messages from this queue. By default, this message stays invisible for 30 seconds. To finish removing the message from the queue, you must also call delete_message.

In [12]:
messages = queue_client.receive_messages()

for message in messages:
    print("Dequeueing message: " + message.content)
    queue_client.delete_message(message.id, message.pop_receipt)

There are two ways you can customize message retrieval from a queue. First, you can get a batch of messages (up to 32). Second, you can set a longer or shorter invisibility timeout, allowing your code more or less time to fully process each message.

In [ ]:
messages = queue_client.receive_messages(messages_per_page=5, visibility_timeout=5*60)

for msg_batch in messages.by_page():
    for msg in msg_batch:
        print("Batch dequeue message: " + msg.content)
        queue_client.delete_message(msg)

### Delete a queue

In [13]:
print("Deleting queue: " + queue_client.queue_name)
queue_client.delete_queue()

Deleting queue: test2
